In [15]:
import boto3
import sys,os
sys.path.append("..")
from ingest import gr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from swampy import structshape as ss

### Ingest school directory dataset from S3

In [16]:
conn = boto3.client('s3')
local_dir_filename = 'directory_all.csv.bak'
if not os.path.exists('./' + local_dir_filename):
    conn.download_file('edu-data-bucket','directory/schools_ccd_directory.csv',local_dir_filename)

In [17]:
df0 = pd.read_csv(local_dir_filename,low_memory=False)

In [19]:
df0.shape, df0.head()

((3381565, 52),
    year      ncessch  ncessch_num  school_id             school_name  \
 0  1986  10000201704  10000201704    1.7E+03       CHALKVILLE CAMPUS   
 1  1986  10000201705  10000201705    1.7E+03         MT MEIGS CAMPUS   
 2  1986  10000201706  10000201706    1.7E+03            VACCA CAMPUS   
 3  1986  10000500870  10000500870  870.0E+00  ALABAMA AVE MIDDLE SCH   
 4  1986  10000500871  10000500871  871.0E+00    ALBERTVILLE HIGH SCH   
 
        leaid                      lea_name state_leaid seasch street_mailing  \
 0  100.0E+03  DEPARTMENT OF YOUTH SERVICES         210   0020            NaN   
 1  100.0E+03  DEPARTMENT OF YOUTH SERVICES         210   0030            NaN   
 2  100.0E+03  DEPARTMENT OF YOUTH SERVICES         210   0040            NaN   
 3  100.0E+03        ALBERTVILLE CITY BOARD         101   0010            NaN   
 4  100.0E+03        ALBERTVILLE CITY BOARD         101   0020            NaN   
 
    ... direct_certification enrollment ungrade_cedp ele